<a href="https://colab.research.google.com/github/bhardwajchetna/Twitter-Sentiment-Analysis---COVID-Vaccinations/blob/main/Twitter_Sentiment_Analysis_Kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kafka & Zookeeper


In [ ]:
!pip install tensorflow-io
!pip install kafka-python

In [ ]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio

In [ ]:
print("tensorflow-io version: {}".format(tfio.__version__))
print("tensorflow version: {}".format(tf.__version__))

tensorflow-io version: 0.17.1
tensorflow version: 2.4.1


In [ ]:
!curl -sSOL https://downloads.apache.org/kafka/2.7.0/kafka_2.13-2.7.0.tgz
!tar -xzf kafka_2.13-2.7.0.tgz

In [ ]:
!./kafka_2.13-2.7.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-2.7.0/config/zookeeper.properties
!./kafka_2.13-2.7.0/bin/kafka-server-start.sh -daemon ./kafka_2.13-2.7.0/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [ ]:
!ps -ef | grep kafka

root         460       1  0 16:41 ?        00:00:26 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.13-2.7.0/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.13-2.7.0/bin/../logs -Dlog4j.configuration=file:./kafka_2.13-2.7.0/bin/../config/log4j.properties -cp /content/kafka_2.13-2.7.0/bin/../libs/activation-1.1.1.jar:/content/kafka_2.13-2.7.0/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.13-2.7.0/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.13-2.7.0/bin/../libs/audience-annotations-0.5.0.jar:/content/kafka_2.13-2.7.0/bin/../libs/commons-cli-1.4.jar:/content/kafka_2.13-2.7.0/bin/../libs/commons-lang3-3.8.1.jar:/content/kafka_2.13

In [ ]:
!./kafka_2.13-2.7.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 3 --topic climatetweets
!./kafka_2.13-2.7.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 3 --topic vaccinationstweets
!./kafka_2.13-2.7.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 3 --topic covidtweets


Created topic climatetweets.
Created topic vaccinationstweets.
Created topic covidtweets.


In [ ]:
!./kafka_2.13-2.7.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic climatetweets
!./kafka_2.13-2.7.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic vaccinationstweets
!./kafka_2.13-2.7.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic covidtweets

Topic: climatetweets	PartitionCount: 3	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: climatetweets	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
	Topic: climatetweets	Partition: 1	Leader: 0	Replicas: 0	Isr: 0
	Topic: climatetweets	Partition: 2	Leader: 0	Replicas: 0	Isr: 0
Topic: vaccinationstweets	PartitionCount: 3	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: vaccinationstweets	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
	Topic: vaccinationstweets	Partition: 1	Leader: 0	Replicas: 0	Isr: 0
	Topic: vaccinationstweets	Partition: 2	Leader: 0	Replicas: 0	Isr: 0
Topic: covidtweets	PartitionCount: 3	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: covidtweets	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
	Topic: covidtweets	Partition: 1	Leader: 0	Replicas: 0	Isr: 0
	Topic: covidtweets	Partition: 2	Leader: 0	Replicas: 0	Isr: 0


### Import Packages and Connect

In [ ]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from kafka import KafkaConsumer, KafkaProducer
import json
import time
import os
import re
from elasticsearch_dsl import Search
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

In [ ]:
ACCESS_TOKEN = ''
ACCESS_SECRET = ''
CONSUMER_KEY = ''
CONSUMER_SECRET = ''

In [ ]:
kafka_producer = KafkaProducer(bootstrap_servers='localhost:9092')

INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: connecting to localhost:9092 [('127.0.0.1', 9092) IPv4]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connected> [IPv4 ('127.0.0.1', 9092)]>: Closing connection. 


### Climate tweets


In [ ]:
count = 0
class StdOutListener(StreamListener):
    def on_data(self, data):
        data_json = json.loads(data)
        #print(data_json.keys())
        global count
        print(count)
        if data_json['user']['followers_count'] > 1000 and count <= 800:
            count = count + 1
            kafka_producer.send("climatetweets", data.encode('utf-8')).get(timeout=3)
            #print(data)
        if count > 800:
          return False
        return True

    def on_error(self, status):
        print(status)


def kafka_run():
    l = StdOutListener()
    auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    stream = Stream(auth, l)
    stream.filter(track=["climate change", "climate", "global warming", "gas", "oil", "sea level", "glaciers", "renewable energy", "CO2", "methane", "ozone", "greenhouse", "environment", "fossil fuel", "green new deal"], languages=["en"])

# Press the green button in the gutter to run the script.
if __name__ == '__main__':
    kafka_run()

### Vaccination tweets

In [ ]:
count1 = 0
class StdOutListener(StreamListener):
    def on_data(self, data):
        data_json = json.loads(data)
        #print(data_json.keys())
        global count1
        print(count1)
        if data_json['user']['followers_count'] > 1000 and count1 <= 800:
            count1 = count1 + 1
            kafka_producer.send("vaccinationstweets", data.encode('utf-8')).get(timeout=3)
            #print(data)
        if count1 > 800:
          return False
        return True

    def on_error(self, status):
        print(status)


def kafka_run():
    l = StdOutListener()
    auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    stream = Stream(auth, l)
    stream.filter(track=["vaccinations", "vaccine", "cdc", "immunization", "herd immunity", "vaccinated"], languages=["en"])

# Press the green button in the gutter to run the script.
if __name__ == '__main__':
    kafka_run()

0
0
1
1
1
2
2
2
2
2
2
2
3
3
3
3
3
4
5
5
6
6
6
7
7
8
8
8
8
8
8
9
10
10
10
11
11
11
11
11
12
13
14
14
15
15
15
15
15
15
15
15
16
17
18
19
19
19
20
20
20
20
20
20
20
20
21
21
21
21
21
21
22
22
22
22
22
23
23
23
23
24
25
25
26
27
28
28
28
28
28
29
30
30
30
31
32
33
34
35
35
36
36
36
36
36
37
37
37
37
37
37
37
37
37
38
38
38
39
39
40
40
41
42
42
42
42
42
43
43
44
44
45
45
45
46
47
48
49
49
50
50
51
51
51
52
53
53
54
55
55
55
55
55
55
55
56
56
56
56
56
57
57
58
58
58
59
59
59
59
59
59
60
60
60
60
60
60
60
60
61
62
62
62
62
63
63
63
63
63
63
63
64
65
65
65
65
65
66
67
67
67
68
68
68
69
69
69
70
70
70
70
70
70
70
71
71
71
72
73
73
73
74
74
74
75
75
76
76
76
76
76
76
76
76
76
77
78
78
79
80
80
80
81
81
82
82
82
82
82
83
84
84
84
85
85
85
85
86
86
86
87
87
87
87
87
88
88
88
88
89
89
89
89
90
90
90
90
90
90
91
92
93
93
93
93
93
93
93
93
94
94
94
94
94
95
95
95
95
95
96
97
97
97
98
98
99
99
99
99
99
99
100
100
100
100
100
101
102
102
102
103
103
104
104
105
105
106
106
106
107
108
109
109
110
110


### COVID tweets

In [ ]:
count2 = 0
class StdOutListener(StreamListener):
    def on_data(self, data):
        data_json = json.loads(data)
        #print(data_json.keys())
        global count2
        print(count2)
        if data_json['user']['followers_count'] > 1000 and count2 <= 800:
            count2 = count2 + 1
            kafka_producer.send("covidtweets", data.encode('utf-8')).get(timeout=3)
            #print(data)
        if count2 > 800:
          return False
        return True

    def on_error(self, status):
        print(status)


def kafka_run():
    l = StdOutListener()
    auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    stream = Stream(auth, l)
    stream.filter(track=["J&J", "Janssen", "Moderna", "Pfizer", "covid", "covid-19", "cdc", "coronavirus"], languages=["en"])

# Press the green button in the gutter to run the script.
if __name__ == '__main__':
    kafka_run()

In [ ]:
#pip install elasticsearch

     |████████████████████████████████| 348kB 6.5MB/s 


### Connect Bonsai

In [ ]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

import os, base64, re, logging
from elasticsearch import Elasticsearch

def elasticsearch_connect():
    # Log transport details (optional):
    logging.basicConfig(level=logging.INFO)
    # Parse the auth and host from env:
    os.environ['BONSAI_URL'] = ''
    bonsai = os.environ['BONSAI_URL']
    auth = re.search('https\:\/\/(.*)\@', bonsai).group(1).split(':')
    host = bonsai.replace('https://%s:%s@' % (auth[0], auth[1]), '')
    
    # optional port
    match = re.search('(:\d+)', host)
    if match:
        p = match.group(0)
        host = host.replace(p, '')
        port = int(p.split(':')[1])
    else:
        port=443
        # Connect to cluster over SSL using auth for best security:
    es_header = [{
        'host': host,
        'port': port,
        'use_ssl': True,
        'http_auth': (auth[0],auth[1])
    }]
    # Instantiate the new Elasticsearch connection:
    es = Elasticsearch(es_header)
    # Verify that Python can talk to Bonsai (optional):
    es.ping()
    
    return es

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Climate to Bonsai

In [ ]:
count1 = 0
def elasticsearch_consumer_main(topicname):
    es = elasticsearch_connect()
    #To Consume Messages
    consumer = KafkaConsumer(topicname, 
                            auto_commit_interval_ms = 30 *1000,
                            auto_offset_reset = 'earliest', 
                            bootstrap_servers=['localhost:9092'])
    #esid = 1
    
    
    global count1
    for message in consumer: 
        #time.sleep(1)
        #print("next")
        #esid +=1
        #if esid % 1000 == 0:
            #print(esid)
            #res_create = es.create(index="tweets", doc_type= topicname, id=1, body={})
        count1 = count1 + 1
        msg = json.loads(message.value)
        
        msg_id = msg['id']
        #print(msg)
        print(count1)
        res = es.index(index = topicname, id = msg_id, body = msg)
        

if __name__ == '__main__':
    elasticsearch_consumer_main('climatetweets')   

INFO:elasticsearch:HEAD https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/ [status:200 request:0.105s]
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: connecting to localhost:9092 [('127.0.0.1', 9092) IPv4]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
INFO:kafka.consumer.subscription_state:Updating subscribed topics to: ('climatetweets',)
INFO:kafka.consumer.subscription_state:Updated partition assignment: [TopicPartition(topic='climatetweets', partition=0), TopicPartition(topic='climatetweets', partition=1), TopicPartition(topic='climatetweets', partition=2)]
INFO:kafka.conn:<BrokerConnection node_id=0 hos

1
2
3
4


INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387902167646887939 [status:201 request:0.028s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387902189255995392 [status:201 request:0.028s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387902195413159937 [status:201 request:0.073s]


5
6
7


INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387902211666022406 [status:201 request:0.124s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387902228938338305 [status:201 request:0.116s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387902244016820225 [status:201 request:0.030s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387902254452117504 [status:201 request:0.028s]


...

INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387904349393719299 [status:201 request:0.028s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387904371036413953 [status:201 request:0.031s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387904380762955776 [status:201 request:0.028s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387904386165219330 [status:201 request:0.027s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387904391127121921 [status:201 request:0.029s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387904391362060288 [status:201 request:0.029s]


790
791
792
793
794
795


INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387904392750411781 [status:201 request:0.027s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387904408654991361 [status:201 request:0.030s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387904418591526912 [status:201 request:0.031s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387904474501378050 [status:201 request:0.028s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387904476904927233 [status:201 request:0.028s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_doc/1387904477437612035 [status:201 request:0.028s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-eas

796
797
798
799
800
801


KeyboardInterrupt: ignored

### Vaccinations to Bonsai

In [ ]:
count1 = 0
def elasticsearch_consumer_main(topicname):
    es = elasticsearch_connect()
    #To Consume Messages
    consumer = KafkaConsumer(topicname, 
                            auto_commit_interval_ms = 30 *1000,
                            auto_offset_reset = 'earliest', 
                            bootstrap_servers=['localhost:9092'])
    #esid = 1
    
    
    global count1
    for message in consumer: 
        #time.sleep(1)
        #print("next")
        #esid +=1
        #if esid % 1000 == 0:
            #print(esid)
            #res_create = es.create(index="tweets", doc_type= topicname, id=1, body={})
        count1 = count1 + 1
        msg = json.loads(message.value)
        
        msg_id = msg['id']
        #print(msg)
        print(count1)
        res = es.index(index = topicname, id = msg_id, body = msg)
        

if __name__ == '__main__':
    elasticsearch_consumer_main('vaccinationstweets')

INFO:elasticsearch:HEAD https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/ [status:200 request:0.081s]
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: connecting to localhost:9092 [('127.0.0.1', 9092) IPv4]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
INFO:kafka.consumer.subscription_state:Updating subscribed topics to: ('vaccinationstweets',)
INFO:kafka.consumer.subscription_state:Updated partition assignment: [TopicPartition(topic='vaccinationstweets', partition=0), TopicPartition(topic='vaccinationstweets', partition=1), TopicPartition(topic='vaccinationstweets', partition=2)]
INFO:kafka.conn:<BrokerConn

1
2
3
4


INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387904509234581515 [status:201 request:0.066s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387904509637169155 [status:201 request:0.070s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387904516113190918 [status:201 request:0.021s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387904521871892482 [status:201 request:0.075s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387904525885837312 [status:201 request:0.020s]


5
6
7
8
9


INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387904528767389698 [status:201 request:0.021s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387904529195167750 [status:201 request:0.023s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387904529144983554 [status:201 request:0.056s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387904530197749760 [status:201 request:0.055s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387904531246243842 [status:201 request:0.021s]


...

INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387905438549790722 [status:201 request:0.022s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387905441468882944 [status:201 request:0.020s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387905446120361988 [status:201 request:0.020s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387905458472587268 [status:201 request:0.019s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387905463241543680 [status:201 request:0.021s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387905466840227842 [status:201 request:0.021s]
INFO:elasticsearch:PUT https://kaf

788
789
790
791
792
793
794
795
796


INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387905494925275136 [status:201 request:0.022s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387905507948744706 [status:201 request:0.022s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387905509261561857 [status:201 request:0.020s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387905510683205632 [status:201 request:0.019s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387905510909775872 [status:201 request:0.021s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_doc/1387905512050626560 [status:201 request:0.020s]


797
798
799
800
801


KeyboardInterrupt: ignored

### COVID to Bonsai

In [ ]:
count1 = 0
def elasticsearch_consumer_main(topicname):
    es = elasticsearch_connect()
    #To Consume Messages
    consumer = KafkaConsumer(topicname, 
                            auto_commit_interval_ms = 30 *1000,
                            auto_offset_reset = 'earliest', 
                            bootstrap_servers=['localhost:9092'])
    #esid = 1
    
    
    global count1
    for message in consumer: 
        #time.sleep(1)
        #print("next")
        #esid +=1
        #if esid % 1000 == 0:
            #print(esid)
            #res_create = es.create(index="tweets", doc_type= topicname, id=1, body={})
        count1 = count1 + 1
        msg = json.loads(message.value)
        
        msg_id = msg['id']
        #print(msg)
        print(count1)
        res = es.index(index = topicname, id = msg_id, body = msg)
        

if __name__ == '__main__':
    elasticsearch_consumer_main('covidtweets')

INFO:elasticsearch:HEAD https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/ [status:200 request:0.109s]
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: connecting to localhost:9092 [('127.0.0.1', 9092) IPv4]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
INFO:kafka.consumer.subscription_state:Updating subscribed topics to: ('covidtweets',)
INFO:kafka.consumer.subscription_state:Updated partition assignment: [TopicPartition(topic='covidtweets', partition=0), TopicPartition(topic='covidtweets', partition=1), TopicPartition(topic='covidtweets', partition=2)]
INFO:kafka.conn:<BrokerConnection node_id=0 host=c8d5f8

1
2
3


INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_doc/1387905521194192896 [status:201 request:0.073s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_doc/1387905524117606402 [status:201 request:0.083s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_doc/1387905524423868416 [status:201 request:0.082s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_doc/1387905525401235456 [status:201 request:0.026s]


4
5
6
7


INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_doc/1387905529490640896 [status:201 request:0.079s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_doc/1387905530635583488 [status:201 request:0.027s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_doc/1387905531495522306 [status:201 request:0.093s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_doc/1387905531998703617 [status:201 request:0.026s]


...

789
790
791
792
793
794
795


INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_doc/1387906207533441029 [status:201 request:0.027s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_doc/1387906207650770950 [status:201 request:0.027s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_doc/1387906213573103616 [status:201 request:0.027s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_doc/1387906221441695744 [status:201 request:0.027s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_doc/1387906221726916616 [status:201 request:0.027s]
INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_doc/1387906223929008137 [status:201 request:0.029s]


796
797
798
799
800
801


INFO:elasticsearch:PUT https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_doc/1387906227036901376 [status:201 request:0.102s]


KeyboardInterrupt: ignored

In [ ]:
def gather_data(topicname):
    es = elasticsearch_connect()
    s = Search(using=es, index=topicname)
    df = pd.DataFrame([hit.to_dict() for hit in s.scan()])

    return df
def identify_tokens(row):
    twitter_text = row['text']
    tokens = nltk.word_tokenize(twitter_text)
    token_words = [w for w in tokens if w.isalpha()]

    return token_words

def stem_list(row):
    stemming = PorterStemmer()
    my_list = row['tokenized']
    stemmed_list = [stemming.stem(word) for word in my_list]

    return stemmed_list

def remove_stops(row):
    stops = set(stopwords.words('english'))
    my_list = row['stemmed']
    meaningful_words = [w for w in my_list if not w in stops]

    return meaningful_words

def rejoin_words(row):
    my_list = row['stopped_removed']
    joined_words = (' '.join(my_list))
    #print(joined_words)

    return joined_words

def text_analytics(doc_type, topicname):
    df = gather_data(topicname)
    df['tokenized'] = df.apply(identify_tokens, axis=1)
    df['stemmed'] = df.apply(stem_list, axis=1)
    df['stopped_removed'] = df.apply(remove_stops, axis=1)
    df.stopped_removed.to_string(index=False)
    df['processed'] = df.apply(rejoin_words, axis=1)
    df.processed.to_string(index=False)

    if (doc_type=="document"):
      df_doc = doc_matrix(df['processed'])
      return df_doc
    elif (doc_type=="tfidf"):
      df_tfidf = tfidf_matrix(df['processed'])
      return df_tfidf

    elif (doc_type=="sentiment"):
      df_sentiment = sentiment(df)
      return df_sentiment

    return False

def doc_matrix(docs, xColNames = None, **kwargs):
    #initialize the  vectorizer
    vectorizer = CountVectorizer()
    x1 = vectorizer.fit_transform(docs)

    #create dataFrame
    df = pd.DataFrame(x1.toarray(), columns = vectorizer.get_feature_names())
    return df

def tfidf_matrix(docs):
    vectorizer = TfidfVectorizer()
    x1 = vectorizer.fit_transform(docs)

    #create dataFrame
    df = pd.DataFrame(x1.toarray(), columns = vectorizer.get_feature_names())
    
    return df

def sentiment(docs):

    sia = SentimentIntensityAnalyzer()

    docs['polarity_scores'] = docs['processed'].map(lambda text: sia.polarity_scores(text)['compound'])
    docs.loc[docs['polarity_scores'] > 0, 'sentiment'] = 'positive' 
    docs.loc[docs['polarity_scores'] < 0, 'sentiment'] = 'negative'
    docs.loc[docs['polarity_scores'] == 0, 'sentiment'] = 'neutral'
    
    return docs

if __name__ == '__main__':
    # create document matrix on climate change
    doc_matrix_climate = text_analytics(doc_type="document", topicname="climatetweets")
    # create TF-IDF matrix on cimate change 
    tfidf_matrix_climate = text_analytics(doc_type="tfidf", topicname="climatetweets")
    # sentiment analysis on vaccinationstweets
    sentiment_matrix_vaccinations = text_analytics(doc_type="sentiment", topicname="vaccinationstweets")
    # sentiment analysis on covidtweets
    sentiment_matrix_covid = text_analytics(doc_type="sentiment", topicname="covidtweets")

INFO:elasticsearch:HEAD https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/ [status:200 request:0.108s]
INFO:elasticsearch:POST https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_search?scroll=5m&size=1000 [status:200 request:1.570s]
INFO:elasticsearch:POST https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/_search/scroll [status:200 request:0.017s]
INFO:elasticsearch:DELETE https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/_search/scroll [status:200 request:0.016s]
INFO:elasticsearch:HEAD https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/ [status:200 request:0.064s]
INFO:elasticsearch:POST https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/climatetweets/_search?scroll=5m&size=1000 [status:200 request:1.321s]
INFO:elasticsearch:POST https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/_search/scroll [status:200 request:0.017s]
INFO:elasticsearch:DELETE https://kafka-h

     aapl   ab  abandon  abc  abe  abiyahmedali  abl  ableist  abnorm  \
0     0.0  0.0      0.0  0.0  0.0           0.0  0.0      0.0     0.0   
1     0.0  0.0      0.0  0.0  0.0           0.0  0.0      0.0     0.0   
2     0.0  0.0      0.0  0.0  0.0           0.0  0.0      0.0     0.0   
3     0.0  0.0      0.0  0.0  0.0           0.0  0.0      0.0     0.0   
4     0.0  0.0      0.0  0.0  0.0           0.0  0.0      0.0     0.0   
..    ...  ...      ...  ...  ...           ...  ...      ...     ...   
796   0.0  0.0      0.0  0.0  0.0           0.0  0.0      0.0     0.0   
797   0.0  0.0      0.0  0.0  0.0           0.0  0.0      0.0     0.0   
798   0.0  0.0      0.0  0.0  0.0           0.0  0.0      0.0     0.0   
799   0.0  0.0      0.0  0.0  0.0           0.0  0.0      0.0     0.0   
800   0.0  0.0      0.0  0.0  0.0           0.0  0.0      0.0     0.0   

     abolish  abov  absolut     abund  abus  aca  acceler  accept  accord  \
0        0.0   0.0      0.0  0.000000   0.0  0

INFO:elasticsearch:POST https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/vaccinationstweets/_search?scroll=5m&size=1000 [status:200 request:1.620s]
INFO:elasticsearch:POST https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/_search/scroll [status:200 request:0.017s]
INFO:elasticsearch:DELETE https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/_search/scroll [status:200 request:0.016s]
INFO:elasticsearch:HEAD https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/ [status:200 request:0.065s]
INFO:elasticsearch:POST https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/covidtweets/_search?scroll=5m&size=1000 [status:200 request:0.836s]
INFO:elasticsearch:POST https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/_search/scroll [status:200 request:0.017s]
INFO:elasticsearch:DELETE https://kafka-homework3-5284633582.us-east-1.bonsaisearch.net:443/_search/scroll [status:200 request:0.016s]


### Climate Top Words

In [ ]:
# Top words
tfidf_matrix_climate.mean().sort_values(ascending=False)[0:20]

rt         0.048665
http       0.045172
climat     0.024663
ga         0.022454
oil        0.021269
thi        0.020958
new        0.019728
environ    0.015538
chang      0.014720
wa         0.014606
polit      0.013868
day        0.013824
job        0.013318
ha         0.012234
biden      0.011413
last       0.011024
amp        0.009728
we         0.009527
citi       0.008967
get        0.008684
dtype: float64

Some of the top words are:


*   climate
*   gas
*   oil
*   new
*   environment

Among the top words, many of them are not very useful based on the topic we had. In addition, many of the top words were words used to filter the tweets. 



There are some advantages of using this method. It successfully removed stop words and the computation was done in a fairly simple way. However, TF-IDF is based on the bag-of-words model, which means it doesn't capture the position in text, semantics, co-occurrences in different documents, etc. 

This is not a reliable method. First of all, most of the top words are simply the keywords we filtered on. There also may be some ambiguity on some of the words since they were tokenized. Lastly, some of the words may be irrelevant like rt (for retweet) and http (for a url).

In [ ]:
# Vaccinations

len(sentiment_matrix_vaccinations.loc[sentiment_matrix_vaccinations['sentiment']=='positive'])/(len(sentiment_matrix_vaccinations.loc[sentiment_matrix_vaccinations['sentiment']=='positive'])+len(sentiment_matrix_vaccinations.loc[sentiment_matrix_vaccinations['sentiment']=='negative']))

0.6813417190775681

In [ ]:
# COVID

len(sentiment_matrix_covid.loc[sentiment_matrix_covid['sentiment']=='positive'])/(len(sentiment_matrix_covid.loc[sentiment_matrix_covid['sentiment']=='positive'])+len(sentiment_matrix_covid.loc[sentiment_matrix_covid['sentiment']=='negative']))

0.567741935483871

Vaccination tweets have a more positive sentiment than COVID vaccination tweets on average. However, this analysis doesn't offer more specific analysis on trending topics within "vaccinationstweets' or " covidtweets". In the future, with more time and data, we recommend building a supervised model for sentiment analysis that could give us more accurate and in-depth results. along with that we can perform topic modeling (supervised or unsupervised) to extract themes prevalent across the data